# Tarea 3 Reinforcement Learning
### (Montecarlo, SARSA, Q-Learning)
Juan Pablo Reyes Fajardo

Santiago Rodríguez Ávila


Para esta tarea se quiere estudiar el comportamiento de los algoritmos de aprendizaje por refuerzo de: Montecarlo, SARSA y Q-learning. Se observarán características, comportamiento y composición de estos. Se hablará de políticas on y off (que el agente sigue la misma política a evaluar o una diferente), todo esto partiendo del escenario de la tarea anterior, el juego de escaleras y serpientes.

## Planteamiento del MDP

En esta parte, se pedía hacer las respectivas modificaciones con el fin de que el módulo de python creado tuviera lo siguiente:
* Dada una acción en un estado, retornar la recompensa y el estado resultante en esa acción. Considere el caso especial del estado terminal.
    
* Que sea posible ejecutar una política arbitraria.

Para esto, se creará un escenario y se utilizará para hacer las pruebas respectivas que se pedían anteriormente. Se procede a crear el escenario:


In [1]:
from types import MethodType
from escalerasyserpientes import EscalerasSerpientes
import copy
import numpy as np

In [2]:
es=EscalerasSerpientes([80,100],[23,37,45,67,89],-0.01)

Luego de crearlo, se mostrará la forma en que el MDP retorna la información luego de realizar un paso, el primer caso será para mostrar la forma de retorno, el segundo va a ser teniendo en cuenta algún estado terminal.

In [3]:
# Muestra de como funciona el MDP
state_prueba=16
action_prueba=5
print(es.step(state_prueba, action_prueba, random=False))

(82, -0.01, 5, False)


La estructura de retorno del MDP es la siguiente:
Casilla de llegada, valor de la recompensa obtenida en el paso, acción tomada en ese estado (valor del dado), variable que muestra si el estado al que se llegó es un estado terminal. En ese orden de ideas, la prueba teniendo en cuenta algún estado terminal sería:

In [4]:
# Muestra de como funciona el MDP en estado terminal
state_prueba=100
action_prueba=-4
print(es.step(state_prueba, action_prueba, random=False))

('Azul', 1.0, None, True)


Aquí se puede ver que cuando el agente llega a un estado terminal e intenta avanzar, el escenario no le permite saltar hacia ningún lado, en este caso, se recibe "Azul"(estado terminal bueno), la recompensa del estado terminal y la variable que informa que se llegó a un estado terminal.

## Algotimo Montecarlo

## Algoritmo SARSA

En esta parte de la tarea se va a analizar y desarrollar el algoritmo SARSA. Como se pudo ver en clase, este es un algoritmo *on-policy*, esto quiere decir que el algoritmo aprende directamente de la política actual del agente. Otra cosa que vale la pena nombrar es que este algoritmo utiliza la **función Q** para actualizar el valor del par estado, acción a evaluar. La norma de actualización de SARSA es la siguiente:

$$Q(S,A) \leftarrow Q(S,A)+\alpha*(R+\gamma*Q(S',A') - Q(S,A))$$

Donde $S'$ es el estado siguiente al que se llega luego de elegir $A$ de forma $\epsilon-greedy$ , y $A'$ es la acción $\epsilon-greedy$ elegida desde el estado $A'$. Finalmente, se repite esta regla de actualización actualizando los valores de forma: $S\leftarrow S'$ y $A\leftarrow A'$ hasta que se llegue a un estado terminal y ahí se termina el episodio.

A continuación, se presenta la implementación del algoritmo:

In [5]:
#SARSA algorithm

# Variables: alpha, gamma y epsilon.
alpha = 0.3
gamma = 1
epsilon = 0.1


#Para 10000 episodios de entrenamiento
for i in range(10000):
    # Inicializa las variables para cada episodio
    num_steps = 0
    state = es.states[0]
    # Seleccion accion "a" de forma epsilon-greedy
    if epsilon< np.random.uniform():        
        act_arg = np.array([es.q_values[(state, act)] for act in es.allowed_actions[state]])
        action = es.allowed_actions[state][np.argmax(act_arg)]
    else:
        actions = es.allowed_actions[state]
        action = actions[np.random.randint(0,2)]
    
    # Inicia el episodio
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]
        
        # Obtengo s'
        new_state, reward, _, done = es.step(state, action, random=True)
        
        # Revisa que new_state no sea un estado terminal
        if type(new_state) is str:
            # Valor q(s',a') terminal
            q_value_next_step = 0
        else:
            #Obtengo a' de s' con epsilon greedy
            if np.random.uniform()< epsilon:     
                actions = es.allowed_actions[new_state]
                new_action = actions[np.random.randint(0,2)]
            else:
                act_arg = np.array([es.q_values[(new_state, act)] for act in es.allowed_actions[state]])
                new_action = es.allowed_actions[new_state][np.argmax(act_arg)]

            # Valor q(s',a') no terminal
            q_value_next_step = es.q_values[(new_state,new_action)]
        
        
        # Calculo de actualizacion q(s,a) <- q(s,a) + alpha*(R + gamma*q(s',a') - q(s,a))
        es.q_values[(state, action)] += alpha*(reward + gamma*q_value_next_step - es.q_values[(state,action)])
        
        # asigna a = a' y s = s'
        state = new_state
        action = new_action
      

        # Parte que termina el episodio si se llega a algun estado terminal
        if done:
            continue_episode = False


Luego de correr la implementación del algoritmo SARSA, la política que se va a utilizar va a ser el valor máximo de la función Q de cada par estado acción por estado. Por ejemplo, la política en el estado 1 va a ser el valor máximo de la **función Q** entre *q(1,"adelante")* y *q(1,"atras")*. 

Teniendo esto en cuenta, la política del algoritmo SARSA es la siguiente:

In [6]:
SARSA_policy = {}
for i in es.states:
    max_val = np.argmax([es.q_values[(i,act)] for act in es.allowed_actions[i]])
    SARSA_policy[i] = es.allowed_actions[i][max_val]
print("Politica obtenida Algoritmo SARSA: ",SARSA_policy)

Politica obtenida Algoritmo SARSA:  {1: 'Ad', 2: 'At', 3: 'Ad', 4: 'Ad', 5: 'Ad', 6: 'Ad', 7: 'Ad', 8: 'Ad', 9: 'Ad', 10: 'Ad', 11: 'Ad', 12: 'Ad', 13: 'Ad', 14: 'Ad', 15: 'Ad', 16: 'Ad', 17: 'At', 18: 'At', 19: 'At', 20: 'At', 21: 'Ad', 22: 'At', 23: 'Ad', 24: 'Ad', 25: 'Ad', 26: 'At', 27: 'Ad', 28: 'Ad', 29: 'Ad', 30: 'Ad', 31: 'At', 32: 'At', 33: 'At', 34: 'At', 35: 'Ad', 36: 'Ad', 37: 'Ad', 38: 'Ad', 39: 'Ad', 40: 'Ad', 41: 'Ad', 42: 'Ad', 43: 'Ad', 44: 'Ad', 45: 'Ad', 46: 'Ad', 47: 'Ad', 48: 'Ad', 49: 'Ad', 50: 'Ad', 51: 'Ad', 52: 'Ad', 53: 'Ad', 54: 'Ad', 55: 'Ad', 56: 'Ad', 57: 'At', 58: 'At', 59: 'Ad', 60: 'At', 61: 'At', 62: 'Ad', 63: 'At', 64: 'Ad', 65: 'At', 66: 'Ad', 67: 'At', 68: 'Ad', 69: 'Ad', 70: 'Ad', 71: 'Ad', 72: 'Ad', 73: 'Ad', 74: 'Ad', 75: 'Ad', 76: 'Ad', 77: 'At', 78: 'Ad', 79: 'Ad', 80: 'Ad', 81: 'At', 82: 'At', 83: 'Ad', 84: 'At', 85: 'At', 86: 'At', 87: 'At', 88: 'At', 89: 'Ad', 90: 'Ad', 91: 'Ad', 92: 'Ad', 93: 'Ad', 94: 'Ad', 95: 'Ad', 96: 'At', 97: 'At', 98

Luego de obtener la política, se procede a hacer una prueba de esta con un total de 1000 episodios. La idea es obtener el promedio de pasos obtenido por cada episodio y el porcentaje de victoria (donde el estado terminal sea "Azul") de los 1000 episodios.

In [7]:
# Prueba de politica en SARSA
num_episodios = 1000
win_per = 0
prom_step = 0

for i in range(num_episodios):
    num_steps = 0
    state = es.states[0]
    states = []
    
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]
            
        # Toma la accion de la politica
        action = SARSA_policy[state]
        # Da un paso en direccion de la politica
        new_state, reward, _, done = es.step(state, action, random=True)
        
        state = new_state
        states.append(state)
        num_steps+=1
        
        if done or num_steps >500:
            continue_episode = False
            if state == 'Azul':
                win_per+=1

    prom_step += 1/(i+1) *(num_steps - prom_step)


El resultado de la prueba de los episodios es la siguiente:

In [8]:
print("Promedio pasos por episodio: ",prom_step)
print("Porcentaje victorias: ", win_per*100/num_episodios)

Promedio pasos por episodio:  40.33399999999996
Porcentaje victorias:  93.0


Si el porcentaje de victorias no aceptable, se puede reiniciar y volver a correr todas las celdas del cuaderno. De esta forma es posible obtener mejores resultados.

Se hizo una prueba de correr 100 veces el cuaderno y observar las recompensas obtenidas. De las 100 veces, se obtuvo un resultado de 80% de resultados favorables, eso quiere decir que probabilísitcamente se puede obtener un buen resultado si se vuelve a correr todo el cuaderno. 

## Algoritmo Q-Learning

En esta parte de la tarea se va a analizar y desarrollar el algoritmo *Q-Learning*. Como se pudo ver en clase, este es un algoritmo *off-policy*, esto quiere decir que el algoritmo aprende de una función de valor que puede usarse para derivar la política óptima en un entorno dado, es decir, no aprende directamente de la política que se está evaluando. Otra cosa que vale la pena nombrar es que este algoritmo también utiliza la **función Q** para actualizar el valor del par estado, acción a evaluar. La norma de actualización de *Q-learning* es la siguiente:

$$Q(S,A) \leftarrow Q(S,A)+\alpha*(R+\gamma*\max_{a}Q(S',a) - Q(S,A))$$


Donde $S'$ es el estado siguiente al que se llega luego de elegir $A$ de forma $\epsilon-greedy$. Luego, se busca la acción con mayor valor del par estado acción $q(S',a)$, donde $a$ puede ser *"Adelante"* o *"Atras"*.

Finalmente, se repite esta regla de actualización actualizando los valores de forma: $S\leftarrow S'$ hasta que se llegue a un estado terminal y ahí se termina el episodio.

A continuación, se presenta la implementación del algoritmo:

In [9]:
#Q-learning algorithm

# Variables: alpha , gamma y epsilon.
alpha = 0.25
gamma = 1
epsilon = 0.1


#Para 1000 episodio
for i in range(10000):
    # Inicializa las variables para cada episodio
    num_steps = 0
    state = es.states[0]
    
    # Inicia el episodio
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]

        # Tomar acción de forma epsilon-greedy
        if np.random.uniform()<epsilon:
            # paso aleatorio
            A = np.random.randint(0,2)
            actions = es.allowed_actions[1]
            action = actions[A]

        else:
            # paso con accion greedy
            act_arg = np.array([es.q_values[(state, act)] for act in es.allowed_actions[state]])
            action = es.allowed_actions[state][np.argmax(act_arg)]
        
        # Obtengo s'
        new_state, reward, _, done = es.step(state, action, random=True)

        # Valor de max q(s',a). Si es terminal el estado, el valor es 0
        if type(new_state) is str:
            # Valor max q(s',a) terminal
            max_q_val = 0
        else:
            # Valor max q(s',a) no terminal
            action_arg = np.array([es.q_values[(new_state, act)] for act in es.allowed_actions[state]])
            new_action = es.allowed_actions[new_state][np.argmax(action_arg)]
            max_q_val = es.q_values[(new_state,new_action)]
        
        # Calculo de actualizacion q(s,a) <- q(s,a) + alpha*(R + gamma*max q(s',a) - q(s,a))
        es.q_values[(state, action)] += alpha*(reward + gamma*max_q_val - es.q_values[(state,action)])
        
        # asigna s = s'
        state = new_state
     

        # Parte que termina el episodio si se llega a algun estado terminal
        if done:
            continue_episode = False

                
    # Promedio de pasos por episodio
    prom_step += 1/(i+1) *(num_steps - prom_step)

In [10]:
Qlearning_policy = {}
for i in es.states:
    #print(i)
    max_val = np.argmax([es.q_values[(i,act)] for act in es.allowed_actions[i]])
    Qlearning_policy[i] = es.allowed_actions[i][max_val]
print("Politica obtenida Algoritmo Q-learning: ",Qlearning_policy)

Politica obtenida Algoritmo Q-learning:  {1: 'At', 2: 'At', 3: 'At', 4: 'Ad', 5: 'Ad', 6: 'At', 7: 'Ad', 8: 'Ad', 9: 'Ad', 10: 'Ad', 11: 'Ad', 12: 'Ad', 13: 'Ad', 14: 'Ad', 15: 'Ad', 16: 'Ad', 17: 'At', 18: 'At', 19: 'At', 20: 'At', 21: 'Ad', 22: 'At', 23: 'Ad', 24: 'Ad', 25: 'At', 26: 'Ad', 27: 'Ad', 28: 'Ad', 29: 'Ad', 30: 'Ad', 31: 'At', 32: 'At', 33: 'At', 34: 'At', 35: 'At', 36: 'At', 37: 'Ad', 38: 'Ad', 39: 'Ad', 40: 'At', 41: 'At', 42: 'Ad', 43: 'Ad', 44: 'Ad', 45: 'Ad', 46: 'Ad', 47: 'Ad', 48: 'Ad', 49: 'Ad', 50: 'Ad', 51: 'Ad', 52: 'Ad', 53: 'Ad', 54: 'Ad', 55: 'Ad', 56: 'At', 57: 'At', 58: 'At', 59: 'Ad', 60: 'Ad', 61: 'At', 62: 'Ad', 63: 'At', 64: 'Ad', 65: 'At', 66: 'Ad', 67: 'Ad', 68: 'Ad', 69: 'Ad', 70: 'Ad', 71: 'Ad', 72: 'Ad', 73: 'Ad', 74: 'Ad', 75: 'Ad', 76: 'Ad', 77: 'Ad', 78: 'Ad', 79: 'At', 80: 'Ad', 81: 'At', 82: 'At', 83: 'Ad', 84: 'At', 85: 'At', 86: 'At', 87: 'At', 88: 'At', 89: 'At', 90: 'Ad', 91: 'Ad', 92: 'Ad', 93: 'Ad', 94: 'Ad', 95: 'Ad', 96: 'At', 97: 'At

In [11]:
# Prueba de polica Qlearning
num_episodios = 1000
win_per = 0
prom_step = 0

for i in range(num_episodios):
    num_steps = 0
    state = es.states[0]
    states = []
    
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]
            
        # Toma la accion de la politica
        action = Qlearning_policy[state]
        # Da un paso en direccion de la politica
        new_state, reward, _, done = es.step(state, action, random=True)
        
        state = new_state
        states.append(state)
        num_steps+=1
        #print(action, state)
        
        if done or num_steps >500:
            continue_episode = False
            if state == 'Azul':
                win_per+=1
    #print(states)
    prom_step += 1/(i+1) *(num_steps - prom_step)

In [12]:
print('Promedio pasos por episodio:',prom_step)
print('Porcentaje victorias:' ,win_per*100/num_episodios)

Promedio pasos por episodio: 54.15599999999998
Porcentaje victorias: 95.3
